In [ ]:
#default_exp learner

# learner
> The main interface for this library

In [ ]:
#export
# We only import .* so all transforms are in namespace
from fastinference_pytorch.transforms.vision import *
from fastinference_pytorch.transforms.data import *
from fastinference_pytorch.utils import to_device, tensor
from fastinference_pytorch.rebuild import make_pipelines, generate_pipeline, load_model, load_data

In [ ]:
#export
from pathlib import Path
from fastcore.utils import store_attr
import torch

## `data_funcs`

`data_funcs` are designed to be quick references to tell your `Learner` how your data is going to come in. Below is an example for grabbing images, however this should be however you expect your data to be coming in:

In [ ]:
#export
def get_image(fn, mode='RGB'): return PILBase.create(fn,mode=mode)

## Learner

In [ ]:
#export
class Learner():
    """
    Similar to a `fastai` learner for inference

    Params:
      > `path`: The exact path to where your data and model is stored, relative to the `cwd`
      > `data_fn`: Filename of your pickled data
      > `model_fn`: Filename of your model
      > `data_func`: A function in which has the ability to grab your data based on some input.
                     The default grabs an image in a location and opens it with Pillow
      > `bs`: The batch size you are wanting to use per inference (this can be tweaked later)
      > `cpu`: Whether to use the CPU or GPU

    Example use:

    learn = Learner('models/data', 'models/model', data_func=image_getter, bs=4, cpu=True)
    """
    def __init__(self, path = Path('.'), data_fn='data', model_fn='model', data_func=None, bs=16, cpu=False):
        data = load_data(path, data_fn)
        self.n_inp = data['n_inp']
        self.pipelines = make_pipelines(data)
        self.after_item = self.pipelines['after_item']
        self.after_batch = self.pipelines['after_batch']
        self.tfm_y = generate_pipeline(data['tfms'], order=False)
        self.model = load_model(path, model_fn, cpu)
        self.device = 'cpu' if cpu else 'cuda'
        store_attr(self, 'data_func,bs')
        self.decode_func = None
        
    def _make_data(self, data):
        "Passes `data` through `after_item` and `after_batch`, splitting into batches"
        self.n_batches = len(data) // self.bs + (0 if len(data)%self.bs == 0 else 1)
        batch,batches = [],[]
        for d in data:
            d = self.data_func(d)
            for tfm in self.after_item: d = tfm(d)
            batch.append(d)
            if len(batch) == self.bs or (len(batches) == self.n_batches - 1 and len(batch) == len(data)):
                batch = torch.stack(batch, dim=0)
                batch = to_device(batch, self.device)
                for tfm in self.after_batch: 
                    batch = tfm(batch)
                batches.append(batch)
                batch = []
        return batches
    
    def _decode_inputs(self, inps, outs):
        "Decodes images through `after_batch`"
        for tfm in self.after_batch[::-1]:
            if hasattr(tfm, 'can_decode'):
                inps = to_device(tfm(inps, decode=True))
        outs.insert(len(outs), inps)
        return outs
    
    def get_preds(self, data, raw_outs=False, decode_func=None, with_input=False):
        """
        Gather predictions on `data` with possible decoding. 
        
        Params:
          > `data`: Incoming data formatted to what `self.data_func`is expecting
          > `raw_outs`: Whether to return the raw outputs
          > `decode_func`: A function to use for decoding potential outputs.
                           While the default is `None`, see `decode_cel` for an example
          > `with_input`: Whether to return a decoded input up to what the model was passed
        """
        inps, outs, dec_out, raw = [],[],[],[]
        batches = self._make_data(data)
        if self.n_inp > 1:
            [inps.append([]) for _ in range(n_inp)]
        with torch.no_grad():
            for batch in batches:
                if with_input:
                    if self.n_inp > 1:
                        for i in range(self.n_inp):
                            inps[i].append(batch[i].cpu())
                        else:
                            inps.append(batch[0].cpu())
                self.model.eval()
                out = self.model(batch[:self.n_inp])
            raw.append(out)
            if self.decode_func is not None: dec_out.append(decode_func(out))
        raw = torch.cat(raw, dim=0).cpu().numpy()
        if self.decode_func is not None: dec_out = torch.cat(dec_out, dim=0)
        if self.decode_func is None or raw_outs: outs.insert(0, raw)
        else: outs.insert(0, raw)
        
        if with_input: outs = self._decode_inputs(inps, outs)
        return outs